### Load all examples
 * `generate_uids=True`: return UIDs per example
 * `tokenizer=None`: return raw (untokenized) examples

In [1]:
from utils import *

In [2]:
from metal.mmtl.utils.preprocess import load_tsv, get_task_tsv_config

In [3]:
config = get_task_tsv_config('COLA', 'dev')
    
(examples, labels), uids = load_tsv(
    tsv_path=config["tsv_path"],
    sent1_idx=config["sent1_idx"],
    sent2_idx=config["sent2_idx"],
    label_idx=config["label_idx"],
    skip_rows=config["skip_rows"],
    tokenizer=None,
    delimiter="\t",
    label_fn=config["label_fn"],
    generate_uids=True
)

assert len(examples) == len(labels) == len(uids)

### Define Proper Nouns based on Entities
Ref: https://spacy.io/api/annotation#named-entities

In [4]:
import spacy
# nlp = spacy.load('en')
nlp = spacy.load('xx_ent_wiki_sm')

ORG_LOC_LABELS = [
    "ORG", "LOC", 
]

def get_proper_nouns(sent):
    if sent is None:
        return []

    return [ent for ent in nlp(sent).ents]

def get_org_loc(sent):
    if sent is None:
        return []

    return [ent for ent in nlp(sent).ents
                if ent.label_ in ORG_LOC_LABELS]

### Tag all Proper Nouns

In [5]:
from tagger import Tagger
tagger = Tagger(verbose=False)

In [6]:
# for idx, (ex, label, uid) in enumerate(zip(examples, labels, uids)):   
#     proper_nouns = get_org_loc(ex['sent1']) \
#         + get_org_loc(ex['sent2'])

#     if idx % 1000 == 0:
#         print(uid)
#         print(ex, label, proper_nouns)
#         print()  

#     if len(proper_nouns) > 0:
#         tagger.add_tag(uid, 'org_loc_nouns')


In [7]:
len(tagger.get_uids("org_loc_nouns"))

61

In [8]:
tagger.get_examples("org_loc_nouns")

[('CoLA/dev.tsv:1027',
  {'sent1': 'I gave it to Pete to take to the fair.',
   'sent2': None,
   'label': '1'}),
 ('CoLA/dev.tsv:124',
  {'sent1': 'No writer, nor any playwright, meets in Vienna.',
   'sent2': None,
   'label': '0'}),
 ('CoLA/dev.tsv:125',
  {'sent1': 'That you will marry any student is not certain.',
   'sent2': None,
   'label': '1'}),
 ('CoLA/dev.tsv:131',
  {'sent1': 'The worker will have a job.', 'sent2': None, 'label': '1'}),
 ('CoLA/dev.tsv:172',
  {'sent1': 'Packages drive easily to New York.',
   'sent2': None,
   'label': '0'}),
 ('CoLA/dev.tsv:173',
  {'sent1': 'The chair pushed.', 'sent2': None, 'label': '0'}),
 ('CoLA/dev.tsv:195',
  {'sent1': 'Ellen said that melons were selling well.',
   'sent2': None,
   'label': '1'}),
 ('CoLA/dev.tsv:200',
  {'sent1': 'Paul laughed at Mary.', 'sent2': None, 'label': '1'}),
 ('CoLA/dev.tsv:23',
  {'sent1': 'Mickey looked up it.', 'sent2': None, 'label': '0'}),
 ('CoLA/dev.tsv:257',
  {'sent1': 'John paid me against t

### Eval on slices

In [9]:
from metal.mmtl.debugging.utils import load_data_and_model, create_dataframe

# Load model and data
# model_path = '/dfs/scratch0/chami/metal/logs/2019_02_20/COLA_23_41_22/best_model.pth'
# model_path = '/dfs/scratch0/mccreery/mmtl/logs/ST_bertlarge/COLA/logdir/2019_02_25/COLA_21_56_02/best_model.pth'
model_path = '/dfs/scratch0/mccreery/mmtl/logs/ST_bertlarge/COLA_cased/2/logdir/2019_03_05/COLA_00_50_04/best_model.pth'
task_name = 'COLA'
split = 'dev'
bert_model = "bert-large-cased"
model,dl = load_data_and_model(model_path, [task_name], split, bert_model=bert_model)

Could not find kwarg "generate_uids" in destination dict.
Using random seed: 321428
Loading COLA Dataset


In [10]:
filepath = f'{task_name}_{split}_error_analysis.tsv'

In [11]:
# Create DataFrame of Raw Data, Predictions, and Labels
print('Creating dataframe')
df_error = create_dataframe(task_name, model, dl, bert_model=bert_model)
print('Created dataframe')

# Save (and reload) DataFrame
csv_path = '/'.join(model_path.split('/')[0:-1])
save_dataframe(df_error, filepath)

Creating dataframe


1042it [00:32, 31.61it/s]


Created dataframe
Saved dataframe to:  COLA_dev_error_analysis.tsv


In [12]:
df_error = load_dataframe(filepath)

In [13]:
df_error.head()

,Unnamed: 0,sentence1,sentence2,score,label,uid,pred,is_wrong
0,0,The weights made the rope stretch over the pu...,NaN,0.999909,1,CoLA/dev.tsv:2,1,False
1,1,The mechanical doll w ##rig ##gled itself loo...,NaN,0.000127,1,CoLA/dev.tsv:3,0,True
2,2,"If you had eaten more , you would want less .",NaN,0.999914,1,CoLA/dev.tsv:4,1,False
3,3,"As you eat the most , you want the least .",NaN,0.999184,0,CoLA/dev.tsv:5,1,True
4,4,"The more you would want , the less you would ...",NaN,0.999899,0,CoLA/dev.tsv:6,1,True


In [14]:
org_loc_uids = tagger.get_uids("org_loc_nouns")
in_slice = df_error['uid'].apply(lambda x: x in org_loc_uids)
df_error[in_slice]

,Unnamed: 0,sentence1,sentence2,score,label,uid,pred,is_wrong
6,6,"Mary listen ##s to the G ##rate ##ful Dead , ...",NaN,0.998953,1,CoLA/dev.tsv:8,1,False
21,21,I said that never in my life had I seen a pla...,NaN,0.999878,1,CoLA/dev.tsv:23,1,False
44,44,Us love they .,NaN,0.000081,0,CoLA/dev.tsv:46,0,False
74,74,The mayor regarded as being absurd the propos...,NaN,0.999908,0,CoLA/dev.tsv:76,1,True
79,79,Himself is understood by Rutherford .,NaN,0.000045,0,CoLA/dev.tsv:81,0,False
87,87,I live at the place where Route 150 crosses t...,NaN,0.999864,1,CoLA/dev.tsv:89,1,False
88,88,I live at the place where Route 150 crosses t...,NaN,0.999800,0,CoLA/dev.tsv:90,1,True
90,90,Wind was gotten of a plot to negotiate an hon...,NaN,0.000057,0,CoLA/dev.tsv:92,0,False
122,122,"No writer , and no playwright , meets in Vien...",NaN,0.999385,0,CoLA/dev.tsv:124,1,True
123,123,"No writer , nor any playwright , meets in Vie...",NaN,0.035777,0,CoLA/dev.tsv:125,0,False


In [15]:
df_wrong_in_slice = df_error[in_slice][df_error[in_slice]['is_wrong']]
df_wrong_in_slice

,Unnamed: 0,sentence1,sentence2,score,label,uid,pred,is_wrong
74,74,The mayor regarded as being absurd the propos...,NaN,0.999908,0,CoLA/dev.tsv:76,1,True
88,88,I live at the place where Route 150 crosses t...,NaN,0.999800,0,CoLA/dev.tsv:90,1,True
122,122,"No writer , and no playwright , meets in Vien...",NaN,0.999385,0,CoLA/dev.tsv:124,1,True
198,198,Cynthia chewed .,NaN,0.999898,0,CoLA/dev.tsv:200,1,True
467,467,Only Churchill remembered Churchill giving th...,NaN,0.999896,0,CoLA/dev.tsv:469,1,True
588,588,The T ##A ' s have been arguing about whether...,NaN,0.999845,0,CoLA/dev.tsv:590,1,True
653,653,Harry told Sue that Albania is a lovely place...,NaN,0.000287,1,CoLA/dev.tsv:655,0,True


In [16]:
print ("Error rate:", len(df_wrong_in_slice) / len(df_error[in_slice]))

Error rate: 0.11475409836065574
